In [1]:
import pandas as pd 
import glob
import os 
#this block of code takes the csv generated from Flight_ETL notebook
# And then grabs the latest one that was saved to the file path
data= "flight_data/raw/"
csv_files = glob.glob(os.path.join(data, "flight_arrivals_*.csv"))

latest_file = max(csv_files,key=os.path.getmtime)

flight_df = pd.read_csv(latest_file)
flight_df.head()

,flight_date,weekday,arrival_iataCode,arrival_terminal,arrival_scheduledTime,departure_iataCode,airline_name,flight_iataNumber,aircraft_modelText
0,2025-08-14,4,lax,3,17:15,sea,air france,af8671,boeing 737-832
1,2025-08-14,4,lax,3,17:15,sea,korean air,ke3455,boeing 737-832
2,2025-08-14,4,lax,3,17:15,sea,sas,sk3644,boeing 737-832
3,2025-08-14,4,lax,3,17:15,sea,saudia,sv6886,boeing 737-832
4,2025-08-14,4,lax,3,17:15,sea,virgin atlantic,vs3213,boeing 737-832


In [2]:
flight_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18986 entries, 0 to 18985
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   flight_date            18986 non-null  object
 1   weekday                18986 non-null  int64 
 2   arrival_iataCode       18986 non-null  object
 3   arrival_terminal       18792 non-null  object
 4   arrival_scheduledTime  18978 non-null  object
 5   departure_iataCode     18951 non-null  object
 6   airline_name           18965 non-null  object
 7   flight_iataNumber      18974 non-null  object
 8   aircraft_modelText     18907 non-null  object
dtypes: int64(1), object(8)
memory usage: 1.3+ MB


In [3]:
#A quick view of different airplane to see what we are work with
sorted(flight_df['airline_name'].dropna().unique())

['abx air',
 'aer lingus',
 'aerologic',
 'aeromexico',
 'air canada',
 'air caraibes',
 'air china',
 'air china cargo',
 'air china ltd',
 'air europa',
 'air france',
 'air incheon',
 'air new zealand',
 'air premia',
 'air serbia',
 'air tahiti nui',
 'air transat',
 'aircalin',
 'alaska air cargo',
 'alaska airlines',
 'alaska airlines (honoring those who serve livery)',
 'alaska airlines (oneworld livery)',
 'alaska airlines (star wars livery)',
 'alaska airlines (tiana’s bayou adventure livery)',
 'alaska horizon',
 'alaska horizon (honoring those who serve livery)',
 'alaska skywest',
 'alitalia',
 'all nippon airways',
 'allegiant air',
 'alphasky',
 'amazon air',
 'american airlines',
 'american airlines (oneworld livery)',
 'american eagle',
 'ana',
 'ana cargo',
 'asiana airlines',
 'asiana cargo',
 'atlas air',
 'austrian',
 'austrian airlines',
 'austrian airlines (johann strauss livery)',
 'avianca',
 'avianca (star alliance livery)',
 'avianca (surf city el salvador liv

In [4]:
# there's 3 different options domestic, international, and cargo we won't need cargo to predict demand for uber
domestic_airlines = [
    'alaska airlines', 'alaska horizon', 'allegiant air', 'american airlines',
    'delta air lines', 'delta connection', 'frontier (flint the wood stork livery)',
    'hawaiian airlines', 'jet linx aviation', 'southwest airlines',
    'spirit airlines', 'united airlines', 'united express'
]

international_airlines = [
    'aeromexico', 'air canada', 'air france', 'air new zealand', 'air tahiti nui',
    'ana', 'british airways', 'brussels airlines', 'china eastern airlines',
    'copa airlines', 'emirates', 'finnair', 'iberia', 'kenya airways', 'klm',
    'korean air', 'latam airlines', 'lufthansa', 'malaysia airlines', 'qantas',
    'qatar airways', 'sa avianca', 'sas', 'turkish airlines', 'virgin atlantic',
    'volaris', 'westjet'
]

cargo_airlines = ['amazon air', 'dhl', 'fedex', 'ups', 'alphasky']

In [5]:
# this function takes the list that was above and classifys if an air line is domestic, international or Cargo
def classify_airline(airline):
    if isinstance(airline, str):
        airline = airline.lower()
        if airline in domestic_airlines:
            return 'Domestic'
        elif airline in international_airlines:
            return 'International'
    return 'Unknown'

flight_df['airline_type'] = flight_df['airline_name'].apply(classify_airline)
flight_df.head()

,flight_date,weekday,arrival_iataCode,arrival_terminal,arrival_scheduledTime,departure_iataCode,airline_name,flight_iataNumber,aircraft_modelText,airline_type
0,2025-08-14,4,lax,3,17:15,sea,air france,af8671,boeing 737-832,International
1,2025-08-14,4,lax,3,17:15,sea,korean air,ke3455,boeing 737-832,International
2,2025-08-14,4,lax,3,17:15,sea,sas,sk3644,boeing 737-832,International
3,2025-08-14,4,lax,3,17:15,sea,saudia,sv6886,boeing 737-832,Unknown
4,2025-08-14,4,lax,3,17:15,sea,virgin atlantic,vs3213,boeing 737-832,International


In [6]:
#then we get rid of cargo for a new dataframe
flight_df_clean = flight_df[flight_df['airline_type'] != 'Cargo']
flight_df_clean = flight_df_clean.dropna(subset=['arrival_scheduledTime'])
flight_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18978 entries, 0 to 18985
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   flight_date            18978 non-null  object
 1   weekday                18978 non-null  int64 
 2   arrival_iataCode       18978 non-null  object
 3   arrival_terminal       18787 non-null  object
 4   arrival_scheduledTime  18978 non-null  object
 5   departure_iataCode     18943 non-null  object
 6   airline_name           18957 non-null  object
 7   flight_iataNumber      18966 non-null  object
 8   aircraft_modelText     18899 non-null  object
 9   airline_type           18978 non-null  object
dtypes: int64(1), object(9)
memory usage: 1.6+ MB


In [7]:
#this is tranforming the schedule landing time into minutes and then into an number so that it can be run through the algorithm
flight_df_clean = flight_df_clean.copy()

flight_df_clean['arrival_time'] = pd.to_datetime(flight_df_clean['arrival_scheduledTime'],format='%H:%M', errors='coerce')
flight_df_clean['arrival_minutes'] = (flight_df_clean['arrival_time'].dt.hour * 60 + flight_df_clean['arrival_time'].dt.minute)

In [8]:
#Here we create another function, this is going to a sign a cluster based of density 
from sklearn.cluster import MeanShift

def run_mean_shift(flight_df_clean, bandwidth, label):
    clustered = []

    for date, group in flight_df_clean.groupby('flight_date'):
        group = group.copy()
        X = group[['arrival_minutes']].values

        if len(X) < 2:
            group[f'cluster_{label}'] = -1
        else:
            ms = MeanShift(bandwidth=bandwidth)
            group[f'cluster_{label}'] = ms.fit_predict(X)

        clustered.append(group)

    return pd.concat(clustered).reset_index(drop=True)

In [9]:
# we run two different cluster window's one with 90 minutes and the other 
df = run_mean_shift(flight_df_clean, bandwidth=90, label='90min')
df = run_mean_shift(df, bandwidth=180, label='180min')

c:\Users\brave\miniconda3\envs\dsa-core\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\brave\miniconda3\envs\dsa-core\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "c:\Users\brave\miniconda3\envs\dsa-core\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\brave\miniconda3\envs\dsa-core\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "c:\Users\brave\miniconda3\envs\dsa-core\Lib\subprocess

In [10]:
df.head()

,flight_date,weekday,arrival_iataCode,arrival_terminal,arrival_scheduledTime,departure_iataCode,airline_name,flight_iataNumber,aircraft_modelText,airline_type,arrival_time,arrival_minutes,cluster_90min,cluster_180min
0,2025-08-14,4,lax,3,17:15,sea,air france,af8671,boeing 737-832,International,1900-01-01 17:15:00,1035,3,1
1,2025-08-14,4,lax,3,17:15,sea,korean air,ke3455,boeing 737-832,International,1900-01-01 17:15:00,1035,3,1
2,2025-08-14,4,lax,3,17:15,sea,sas,sk3644,boeing 737-832,International,1900-01-01 17:15:00,1035,3,1
3,2025-08-14,4,lax,3,17:15,sea,saudia,sv6886,boeing 737-832,Unknown,1900-01-01 17:15:00,1035,3,1
4,2025-08-14,4,lax,3,17:15,sea,virgin atlantic,vs3213,boeing 737-832,International,1900-01-01 17:15:00,1035,3,1


In [11]:
def summarize_clusters(df, cluster_col, top_n=3, source='flight'):
    # Rename for consistency
    df = df.copy()
    df = df.rename(columns={cluster_col: 'cluster_id'})

    # Group and summarize
    summary = df.groupby(['flight_date', 'cluster_id','arrival_iataCode']).agg(
        start_min=('arrival_minutes', 'min'),
        end_min=('arrival_minutes', 'max'),
        flight_count=('arrival_minutes', 'count')
    ).reset_index()

    # Add readable times
    summary['start_time'] = pd.to_datetime(summary['start_min'], unit='m').dt.strftime('%H:%M')
    summary['end_time'] = pd.to_datetime(summary['end_min'], unit='m').dt.strftime('%H:%M')
    summary['duration_min'] = summary['end_min'] - summary['start_min']
    summary['midpoint_min'] = (summary['start_min'] + summary['end_min']) // 2
    summary['midpoint_time'] = pd.to_datetime(summary['midpoint_min'], unit='m').dt.strftime('%H:%M')
    # Calculate duration


    # Rank and filter
    summary['rank'] = summary.groupby('flight_date')['flight_count'].rank(ascending=False, method='first')
    top_clusters = summary[summary['rank'] <= top_n].copy()
    top_clusters['source'] = source

    return top_clusters

In [12]:
#taking the summarization funcation and applying it to the dataframe
top_90 = summarize_clusters(df, cluster_col='cluster_90min', top_n=3, source='flight_90min')
top_180 = summarize_clusters(df, cluster_col='cluster_180min', top_n=3, source='flight_180min')

In [13]:
#then we take the to clusters dataframe the we summarized and we append the data
all_top_clusters = pd.concat([top_90, top_180], ignore_index=True)
all_top_clusters.head()

,flight_date,cluster_id,arrival_iataCode,start_min,end_min,flight_count,start_time,end_time,duration_min,midpoint_min,midpoint_time,rank,source
0,2025-08-14,0,lax,335,645,719,05:35,10:45,310,490,08:10,1.0,flight_90min
1,2025-08-14,1,lax,646,805,487,10:46,13:25,159,725,12:05,3.0,flight_90min
2,2025-08-14,2,lax,1205,1439,632,20:05,23:59,234,1322,22:02,2.0,flight_90min
3,2025-08-15,0,lax,335,645,720,05:35,10:45,310,490,08:10,1.0,flight_90min
4,2025-08-15,1,lax,646,800,450,10:46,13:20,154,723,12:03,3.0,flight_90min


In [14]:
all_top_clusters.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   flight_date       42 non-null     object 
 1   cluster_id        42 non-null     int64  
 2   arrival_iataCode  42 non-null     object 
 3   start_min         42 non-null     int32  
 4   end_min           42 non-null     int32  
 5   flight_count      42 non-null     int64  
 6   start_time        42 non-null     object 
 7   end_time          42 non-null     object 
 8   duration_min      42 non-null     int32  
 9   midpoint_min      42 non-null     int32  
 10  midpoint_time     42 non-null     object 
 11  rank              42 non-null     float64
 12  source            42 non-null     object 
dtypes: float64(1), int32(4), int64(2), object(6)
memory usage: 3.7+ KB


In [15]:
# we then write the data to a csv so it can be later ingested into a dashboard
from datetime import datetime, timedelta 
timestamp = datetime.now().strftime('%Y%m%d')
raw_path = f"flight_data/modeled/Top_clusters{timestamp}.parquet"
all_top_clusters.to_parquet(raw_path, index=False)